# Niveau 1 - Détection Heuristique Enrichie

**Objectif principal :** Enrichir l'information de tous les résumés pour alimenter les niveaux suivants (pas de rejet définitif)

## Analyses intégrées (8 types) :
1. Anomalies statistiques (longueur, ponctuation, diversité lexicale)
2. Complexité syntaxique (structure phrases, connecteurs logiques)
3. Répétitions agressives (patterns répétitifs suspects)
4. Densité d'entités (distribution éléments nommés)
5. Incohérences temporelles (anachronismes, contradictions chronologiques)
6. Validation des entités (NER + bases externes)
7. Relations causales suspectes (plausibilité liens cause-effet)
8. Intégration métriques existantes (coherence/factuality scores, grades)



## 1. Configuration et Imports

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
import time
import warnings
import json
warnings.filterwarnings('ignore')

# Configuration des chemins
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

print(f"Répertoire projet: {project_root}")
print(f"Répertoire src: {src_path}")

# Import des modules de détection ENHANCED avec fallback compatibilité
try:
    # Version enhanced avec patterns corrigés et seuils calibrés
    from detection.level0_prefilter_enhanced import EnhancedQualityFilter as QualityFilter
    print("NIVEAU 0 ENHANCED importé")
except ImportError:
    from detection.level0_prefilter import QualityFilter
    print("Niveau 0 original importé")

try:
    # Version enhanced avec seuils corrigés et détection confidence_weighted
    from detection.level1_heuristic import HeuristicAnalyzer as Level1HeuristicDetector
    
    # Alias de compatibilité pour les fonctions rapides
    def quick_heuristic_check(text, metadata=None):
        analyzer = Level1HeuristicDetector(enable_wikidata=False)
        result = analyzer.analyze_summary(text, metadata or {})
        return result.is_suspect, result.issues
    
    # Patch pour compatibilité: ajouter les méthodes manquantes
    original_class = Level1HeuristicDetector
    class CompatLevel1HeuristicDetector(original_class):
        def process_batch(self, summaries, enable_progress=True):
            """Alias de compatibilité pour analyze_batch"""
            return self.analyze_batch(summaries, enable_progress)
            
        def detect_hallucinations(self, text, metadata=None):
            """Alias de compatibilité pour analyze_summary"""
            result = self.analyze_summary(text, metadata or {})
            # Convertir vers format attendu par le notebook
            return {
                'is_hallucination': result.is_suspect,
                'confidence_score': result.confidence_score,
                'issues': result.issues,
                'word_count': result.word_count,
                'processing_time_ms': result.processing_time_ms,
                'risk_level': result.risk_level,
                'corrections_suggested': result.corrections_suggested
            }
    
    Level1HeuristicDetector = CompatLevel1HeuristicDetector
    
    print("NIVEAU 1 ENHANCED importé (analyseur amélioré avec aliases)")
except ImportError:
    from detection.level1_heuristic_original import Level1HeuristicDetector, quick_heuristic_check
    print("Niveau 1 original importé")

print("\nModules importés avec succès")

Répertoire projet: c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector
Répertoire src: c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\src
NIVEAU 0 ENHANCED importé
NIVEAU 1 ENHANCED importé (analyseur amélioré avec aliases)

Modules importés avec succès


## 2. Chargement des Données Complètes

In [2]:
# Charger les 372 résumés originaux
data_path = os.path.join(project_root, 'data', 'results', 'batch_summary_production.csv')
df_original = pd.read_csv(data_path)

print(f"Données originales: {len(df_original)} résumés")

# Analyse de la distribution initiale
initial_grades = Counter(df_original['quality_grade'])
print(f"\nDistribution des grades (données complètes):")
for grade, count in sorted(initial_grades.items()):
    print(f"  Grade {grade}: {count} résumés ({count/len(df_original)*100:.1f}%)")

# Charger les résultats du Niveau 0 pour identifier les 2 populations
level0_results_path = os.path.join(project_root, 'data', 'detection', 'level0_filter_results.csv')
if os.path.exists(level0_results_path):
    df_level0 = pd.read_csv(level0_results_path)
    valid_level0_ids = df_level0[df_level0['filter_valid'] == True]['id'].tolist()
    
    print(f"\nAnalyse Niveau 0:")
    print(f"  Résumés validés par Niveau 0: {len(valid_level0_ids)}")
    print(f"  Résumés rejetés par Niveau 0: {len(df_original) - len(valid_level0_ids)}")
else:
    print("\nRésultats Niveau 0 non trouvés, traitement de tous les résumés")
    valid_level0_ids = None

Données originales: 372 résumés

Distribution des grades (données complètes):
  Grade A: 62 résumés (16.7%)
  Grade A+: 60 résumés (16.1%)
  Grade B: 11 résumés (3.0%)
  Grade B+: 158 résumés (42.5%)
  Grade C: 17 résumés (4.6%)
  Grade D: 64 résumés (17.2%)

Analyse Niveau 0:
  Résumés validés par Niveau 0: 189
  Résumés rejetés par Niveau 0: 183


## 3. Préparation des Données pour Analyse 

In [3]:
# Préparer TOUS les résumés (319 validés + 53 rejetés par Niveau 0)
all_summaries_data = []
level0_validated_data = []
level0_rejected_data = []

for idx, row in df_original.iterrows():
    summary_id = f"{row['text_id']}_{row['fusion_strategy']}"
    
    summary_dict = {
        'id': summary_id,
        'text': row['summary'],
        'original_length': row['length'],
        'quality_grade': row['quality_grade'],
        'coherence': row['coherence'],
        'factuality': row['factuality'],
        'composite_score': row['composite_score'],
        'level0_status': 'validated' if (valid_level0_ids is None or summary_id in valid_level0_ids) else 'rejected'
    }
    
    all_summaries_data.append(summary_dict)
    
    # Séparer en 2 populations pour analyse
    if valid_level0_ids is None or summary_id in valid_level0_ids:
        level0_validated_data.append(summary_dict)
    else:
        level0_rejected_data.append(summary_dict)

print(f"\nDonnées préparées pour analyse Niveau 1:")
print(f"  Total: {len(all_summaries_data)} résumés")
print(f"  Validés par Niveau 0: {len(level0_validated_data)} résumés")
print(f"  Rejetés par Niveau 0: {len(level0_rejected_data)} résumés")



Données préparées pour analyse Niveau 1:
  Total: 372 résumés
  Validés par Niveau 0: 189 résumés
  Rejetés par Niveau 0: 183 résumés


In [4]:

# Distribution par statut Niveau 0
print(f"\nDistribution par population:")
for status in ['validated', 'rejected']:
    subset = [s for s in all_summaries_data if s['level0_status'] == status]
    if subset:
        status_grades = Counter([s['quality_grade'] for s in subset])
        print(f"  Population {status}:")
        for grade, count in sorted(status_grades.items()):
            print(f"    Grade {grade}: {count} résumés")


Distribution par population:
  Population validated:
    Grade A: 62 résumés
    Grade A+: 58 résumés
    Grade B: 7 résumés
    Grade B+: 39 résumés
    Grade D: 23 résumés
  Population rejected:
    Grade A+: 2 résumés
    Grade B: 4 résumés
    Grade B+: 119 résumés
    Grade C: 17 résumés
    Grade D: 41 résumés


## 4. 

In [5]:
# Prendre un exemple représentatif (Grade A+)
test_summary = level0_validated_data[10] if len(level0_validated_data) > 10 else all_summaries_data[10]
detector_enriched = Level1HeuristicDetector(enable_wikidata=False, enable_entity_validation=True)

# Préparer les métadonnées
metadata = {
    'id': test_summary['id'],
    'coherence_score': test_summary['coherence'],
    'factuality_score': test_summary['factuality'],
    'quality_grade': test_summary['quality_grade']
}

result_enriched = detector_enriched.analyze_summary(test_summary['text'], metadata)

print(f"Résumé testé: {test_summary['id']} (Grade: {test_summary['quality_grade']})")
print(f"   Statut Niveau 0: {test_summary['level0_status']}")
print(f"   Extrait: {test_summary['text'][:150]}...")

Résumé testé: 10_adaptive (Grade: A+)
   Statut Niveau 0: validated
   Extrait: , Un portrait défiguré de l’ex président syrien Bachar al-Assad dans un centre de sécurité gouvernemental saccagé, à Damas, le, le jour du renversemen...


In [6]:
print(f"Évaluation globale:")
print(f"   Statut: {'SUSPECT' if result_enriched.is_suspect else 'VALIDE'}")
print(f"   Confiance: {result_enriched.confidence_score:.3f}")
print(f"   Niveau de risque: {result_enriched.risk_level}")
print(f"   Issues détectées: {len(result_enriched.issues)}")
print(f"   Temps de traitement: {result_enriched.processing_time_ms:.1f}ms\n")

Évaluation globale:
   Statut: VALIDE
   Confiance: 0.700
   Niveau de risque: low
   Issues détectées: 1
   Temps de traitement: 82.1ms



In [7]:
# Need to restart the kernel and reimport to get the fixed methods
import importlib
import sys

# Reload the module
if 'detection.level1_heuristic' in sys.modules:
    importlib.reload(sys.modules['detection.level1_heuristic'])

from detection.level1_heuristic import HeuristicAnalyzer

# Re-run the test
test_summary = level0_validated_data[10] if len(level0_validated_data) > 10 else all_summaries_data[10]
detector_enriched = HeuristicAnalyzer(enable_wikidata=False, enable_entity_validation=True)

# Préparer les métadonnées
metadata = {
    'id': test_summary['id'],
    'coherence_score': test_summary['coherence'],
    'factuality_score': test_summary['factuality'],
    'quality_grade': test_summary['quality_grade']
}

result_enriched = detector_enriched.analyze_summary(test_summary['text'], metadata)

print(f"PROFIL STATISTIQUE:")
# Vérifier les propriétés disponibles
print(f"   Nombre de mots: {result_enriched.word_count}")
print(f"   Entités détectées: {result_enriched.entities_detected}")
print(f"   Entités suspectes: {result_enriched.suspicious_entities}")
print(f"   Candidats fact-check: {len(result_enriched.fact_check_candidates)}")
print(f"   Score priorité: {result_enriched.priority_score:.3f}")

PROFIL STATISTIQUE:
   Nombre de mots: 27
   Entités détectées: 3
   Entités suspectes: 0
   Candidats fact-check: 1
   Score priorité: 0.000


In [8]:
print(f"\nEXTRACTION D'ENTITÉS:")
print(f"   Total entités détectées: {result_enriched.entities_detected}")
print(f"   Entités suspectes: {result_enriched.suspicious_entities}")

# Vérifier si les détails sont dans enrichment_metadata
if 'entity_extraction' in result_enriched.enrichment_metadata:
    entities = result_enriched.enrichment_metadata['entity_extraction']
    print(f"   Détails disponibles dans enrichment_metadata:")
    
    if 'persons' in entities:
        persons = entities['persons'][:3] if isinstance(entities['persons'], list) else []
        print(f"   Personnes: {len(entities.get('persons', []))} → {[p.get('text', p) if isinstance(p, dict) else str(p) for p in persons]}")
    
    if 'organizations' in entities:
        orgs = entities['organizations'][:3] if isinstance(entities['organizations'], list) else []
        print(f"   Organisations: {len(entities.get('organizations', []))} → {[o.get('text', o) if isinstance(o, dict) else str(o) for o in orgs]}")
    
    if 'locations' in entities:
        locs = entities['locations'][:3] if isinstance(entities['locations'], list) else []
        print(f"   Lieux: {len(entities.get('locations', []))} → {[l.get('text', l) if isinstance(l, dict) else str(l) for l in locs]}")
else:
    print(f"   (Détails d'entités non disponibles dans enrichment_metadata)")
    print(f"   Clés disponibles: {list(result_enriched.enrichment_metadata.keys())}")


EXTRACTION D'ENTITÉS:
   Total entités détectées: 3
   Entités suspectes: 0
   (Détails d'entités non disponibles dans enrichment_metadata)
   Clés disponibles: ['word_count', 'sentence_count', 'strategy', 'has_corruption', 'repetition_ratio', 'encoding_quality', 'entity_density', 'fact_check_density']


In [9]:
print(f"\nMÉTRIQUES DE COMPLEXITÉ:")

# Vérifier si les métriques sont dans enrichment_metadata
if 'complexity_metrics' in result_enriched.enrichment_metadata:
    complexity_metrics = result_enriched.enrichment_metadata['complexity_metrics']
    for key, value in complexity_metrics.items():
        print(f"   {key}: {value}")
else:
    print(f"   (Métriques de complexité non disponibles dans enrichment_metadata)")
    print(f"   Métadonnées disponibles: {list(result_enriched.enrichment_metadata.keys())}")
    
    # Afficher les métriques de base disponibles
    print(f"   Métriques de base disponibles:")
    print(f"     Nombre de mots: {result_enriched.word_count}")
    print(f"     Score de priorité: {result_enriched.priority_score:.3f}")
    print(f"     Nombre d'issues: {len(result_enriched.issues)}")
    print(f"     Temps de traitement: {result_enriched.processing_time_ms:.1f}ms")


MÉTRIQUES DE COMPLEXITÉ:
   (Métriques de complexité non disponibles dans enrichment_metadata)
   Métadonnées disponibles: ['word_count', 'sentence_count', 'strategy', 'has_corruption', 'repetition_ratio', 'encoding_quality', 'entity_density', 'fact_check_density']
   Métriques de base disponibles:
     Nombre de mots: 27
     Score de priorité: 0.000
     Nombre d'issues: 1
     Temps de traitement: 15.7ms


In [10]:
print(f"\n4. INDICATEURS DE QUALITÉ:")

# Vérifier si les indicateurs sont dans enrichment_metadata
if 'quality_indicators' in result_enriched.enrichment_metadata:
    quality_indicators = result_enriched.enrichment_metadata['quality_indicators']
    for key, value in quality_indicators.items():
        print(f"   {key}: {value}")
else:
    print(f"   (Indicateurs de qualité non disponibles dans enrichment_metadata)")
    
    # Afficher les indicateurs de base disponibles
    print(f"   Indicateurs de base disponibles:")
    print(f"     Statut: {'SUSPECT' if result_enriched.is_suspect else 'VALIDE'}")
    print(f"     Confiance: {result_enriched.confidence_score:.3f}")
    print(f"     Niveau de risque: {result_enriched.risk_level}")
    print(f"     Score de priorité: {result_enriched.priority_score:.3f}")
    
    if result_enriched.severity_breakdown:
        print(f"     Répartition sévérité: {result_enriched.severity_breakdown}")
    
    if result_enriched.corrections_suggested:
        print(f"     Corrections suggérées: {len(result_enriched.corrections_suggested)} items")


4. INDICATEURS DE QUALITÉ:
   (Indicateurs de qualité non disponibles dans enrichment_metadata)
   Indicateurs de base disponibles:
     Statut: VALIDE
     Confiance: 0.700
     Niveau de risque: low
     Score de priorité: 0.000
     Répartition sévérité: {'critical': 0, 'moderate': 1, 'minor': 0}


In [11]:
print(f"\nCANDIDATS FACT-CHECK (Top 5):")

if result_enriched.fact_check_candidates:
    candidates = result_enriched.fact_check_candidates[:5]
    print(f"   Nombre total de candidats: {len(result_enriched.fact_check_candidates)}")
    
    for i, candidate in enumerate(candidates, 1):
        if isinstance(candidate, dict):
            # Utiliser la vraie structure: type, priority, description
            type_val = candidate.get('type', 'N/A')
            priority_val = candidate.get('priority', 'N/A')
            description_val = candidate.get('description', 'N/A')
            
            print(f"   {i}. Type: {type_val} | Priorité: {priority_val} | Description: {description_val}")
        else:
            print(f"   {i}. Candidat: {candidate}")
else:
    print(f"   Aucun candidat fact-check détecté")


CANDIDATS FACT-CHECK (Top 5):
   Nombre total de candidats: 1
   1. Type: entity_verification | Priorité: 0.7 | Description: Vérification des 3 entités détectées


In [12]:
# ===== ANALYSE AVEC CODE CORRIGÉ =====
# Force reload du module corrigé
import importlib
if 'detection.level1_heuristic' in sys.modules:
    importlib.reload(sys.modules['detection.level1_heuristic'])

from detection.level1_heuristic import HeuristicAnalyzer

print("Module rechargé avec les corrections appliquées")
print("Corrections :")
print("- Seuils plus permissifs (0.3 → 0.05)")
print("- Pénalités réduites (70% → 15-30%)")
print("- Enrichissement proactif") 
print("- Validation externe désactivée par défaut")

detector_main = HeuristicAnalyzer(enable_wikidata=False, enable_entity_validation=True)

print(f"\nTraitement en cours des {len(level0_validated_data)} résumés validés Niveau 0...")

start_time = time.time()

# Traitement batch manuel car process_batch n'existe peut-être pas
all_results_main = []
valid_summaries_main = []

for i, summary_data in enumerate(level0_validated_data):
    # Préparer les métadonnées
    metadata = {
        'id': summary_data['id'],
        'coherence_score': summary_data['coherence'],
        'factuality_score': summary_data['factuality'],
        'quality_grade': summary_data['quality_grade']
    }
    
    # Analyser le résumé
    result = detector_main.analyze_summary(summary_data['text'], metadata)
    all_results_main.append(result)
    
    # Ajouter aux valides si pas suspect
    if not result.is_suspect:
        valid_summaries_main.append(summary_data)
    
    # Progress indicator
    if (i + 1) % 50 == 0:
        print(f"   Traité {i + 1}/{len(level0_validated_data)} résumés...")

total_time_main = time.time() - start_time

print(f"Traitement terminé en {total_time_main:.2f}s")
print(f"\nRésultats sur résumés validés Niveau 0 (APRÈS CORRECTIONS):")
print(f"   Total analysé: {len(level0_validated_data)}")
print(f"   Suspects détectés: {len(level0_validated_data) - len(valid_summaries_main)}")
print(f"   Taux de détection: {(len(level0_validated_data) - len(valid_summaries_main))/len(level0_validated_data)*100:.1f}%")
print(f"   Temps moyen: {total_time_main/len(level0_validated_data)*1000:.1f}ms par résumé")

Module rechargé avec les corrections appliquées
Corrections :
- Seuils plus permissifs (0.3 → 0.05)
- Pénalités réduites (70% → 15-30%)
- Enrichissement proactif
- Validation externe désactivée par défaut

Traitement en cours des 189 résumés validés Niveau 0...
   Traité 50/189 résumés...
   Traité 100/189 résumés...
   Traité 150/189 résumés...
Traitement terminé en 9.52s

Résultats sur résumés validés Niveau 0 (APRÈS CORRECTIONS):
   Total analysé: 189
   Suspects détectés: 16
   Taux de détection: 8.5%
   Temps moyen: 50.4ms par résumé


In [13]:
# Vérification objectif performance (APRÈS CORRECTIONS)
processing_times_main = [r.processing_time_ms for r in all_results_main]
target_met = sum(1 for t in processing_times_main if t < 100) / len(processing_times_main) * 100
print(f"   Objectif <100ms: {target_met:.1f}% des résumés")

# Amélioration vs objectif original (49.5%)
improvement = target_met - 49.5
print(f"   Amélioration performance: {improvement:+.1f} points")

# Nouvelles métriques d'enrichissement
total_fact_check_new = sum(len(r.fact_check_candidates) for r in all_results_main)

# Vérifier si validation_hints existe dans enrichment_metadata
total_wikidata_new = 0
for r in all_results_main:
    if 'validation_hints' in r.enrichment_metadata:
        validation_hints = r.enrichment_metadata['validation_hints']
        if isinstance(validation_hints, dict) and 'wikidata_queries' in validation_hints:
            total_wikidata_new += len(validation_hints['wikidata_queries'])

print(f"   Candidats fact-check générés: {total_fact_check_new} (vs 5 avant)")
print(f"   Requêtes Wikidata: {total_wikidata_new} (vs 5 avant)")

# Debug: Afficher la structure pour comprendre
if all_results_main:
    first_result = all_results_main[0]
    print(f"   (Debug) Clés enrichment_metadata: {list(first_result.enrichment_metadata.keys())}")

   Objectif <100ms: 91.0% des résumés
   Amélioration performance: +41.5 points
   Candidats fact-check générés: 194 (vs 5 avant)
   Requêtes Wikidata: 0 (vs 5 avant)
   (Debug) Clés enrichment_metadata: ['word_count', 'sentence_count', 'strategy', 'has_corruption', 'repetition_ratio', 'encoding_quality', 'entity_density', 'fact_check_density']


In [14]:
# Corrélation avec grades de qualité (APRÈS CORRECTIONS)
print(f"\nCorrélation avec grades de qualité (NOUVELLES DONNÉES):")
print(f"{'Grade':<8} {'Suspects':<12} {'%':<8} {'Confiance':<12} {'Temps (ms)'}")
print("-" * 55)

for grade in ['A+', 'A', 'B+', 'B', 'C', 'D']:
    grade_data = [s for s in level0_validated_data if s['quality_grade'] == grade]
    if grade_data:
        grade_results = [all_results_main[i] for i, s in enumerate(level0_validated_data) if s['quality_grade'] == grade]
        suspect_count = sum(1 for r in grade_results if r.is_suspect)  # CORRIGÉ: is_suspect au lieu de not is_valid
        total_count = len(grade_data)
        avg_confidence = sum(r.confidence_score for r in grade_results) / len(grade_results)
        avg_time = sum(r.processing_time_ms for r in grade_results) / len(grade_results)
        
        print(f"{grade:<8} {suspect_count}/{total_count:<8} {suspect_count/total_count*100:<8.1f} {avg_confidence:<12.3f} {avg_time:.1f}")

# Comparaison avant/après pour les grades critiques
print(f"\n=== COMPARAISON AVANT/APRÈS CORRECTIONS ===")
a_plus_suspects = sum(1 for i, s in enumerate(level0_validated_data) if s['quality_grade'] == 'A+' and all_results_main[i].is_suspect)
a_plus_total = sum(1 for s in level0_validated_data if s['quality_grade'] == 'A+')
d_suspects = sum(1 for i, s in enumerate(level0_validated_data) if s['quality_grade'] == 'D' and all_results_main[i].is_suspect)
d_total = sum(1 for s in level0_validated_data if s['quality_grade'] == 'D')

print(f"Grade A+ suspects: AVANT 0/{a_plus_total} (0.0%) → APRÈS {a_plus_suspects}/{a_plus_total} ({a_plus_suspects/a_plus_total*100:.1f}%)")
print(f"Grade D suspects: AVANT {d_total}/{d_total} (100.0%) → APRÈS {d_suspects}/{d_total} ({d_suspects/d_total*100:.1f}%)")

# Sauvegarder les nouveaux résultats
globals()['level0_validated_results'] = all_results_main


Corrélation avec grades de qualité (NOUVELLES DONNÉES):
Grade    Suspects     %        Confiance    Temps (ms)
-------------------------------------------------------
A+       1/58       1.7      0.873        47.4
A        2/62       3.2      0.837        53.9
B+       3/39       7.7      0.763        52.8
B        3/7        42.9     0.521        63.7
D        7/23       30.4     0.765        39.1

=== COMPARAISON AVANT/APRÈS CORRECTIONS ===
Grade A+ suspects: AVANT 0/58 (0.0%) → APRÈS 1/58 (1.7%)
Grade D suspects: AVANT 23/23 (100.0%) → APRÈS 7/23 (30.4%)


In [15]:
detector_main = HeuristicAnalyzer(enable_wikidata=False, enable_entity_validation=True)

print(f"Traitement en cours des {len(level0_validated_data)} résumés validés Niveau 0...")

start_time = time.time()

# Traitement batch manuel
all_results_main = []
valid_summaries_main = []

for i, summary_data in enumerate(level0_validated_data):
    # Préparer les métadonnées
    metadata = {
        'id': summary_data['id'],
        'coherence_score': summary_data['coherence'],
        'factuality_score': summary_data['factuality'],
        'quality_grade': summary_data['quality_grade']
    }
    
    # Analyser le résumé
    result = detector_main.analyze_summary(summary_data['text'], metadata)
    all_results_main.append(result)
    
    # Ajouter aux valides si pas suspect
    if not result.is_suspect:
        valid_summaries_main.append(summary_data)

total_time_main = time.time() - start_time

print(f"Traitement terminé en {total_time_main:.2f}s")
print(f"\nRésultats sur résumés validés Niveau 0:")
print(f"   Total analysé: {len(level0_validated_data)}")
print(f"   Suspects détectés: {len(level0_validated_data) - len(valid_summaries_main)}")
print(f"   Taux de détection: {(len(level0_validated_data) - len(valid_summaries_main))/len(level0_validated_data)*100:.1f}%")
print(f"   Temps moyen: {total_time_main/len(level0_validated_data)*1000:.1f}ms par résumé")

Traitement en cours des 189 résumés validés Niveau 0...
Traitement terminé en 7.77s

Résultats sur résumés validés Niveau 0:
   Total analysé: 189
   Suspects détectés: 16
   Taux de détection: 8.5%
   Temps moyen: 41.1ms par résumé


In [16]:
# ===== ANALYSE RÉSUMÉS REJETÉS NIVEAU 0 (AVEC CODE CORRIGÉ) =====

if len(level0_rejected_data) > 0:
    # Utiliser le même détecteur corrigé
    detector_rejected = HeuristicAnalyzer(enable_wikidata=False, enable_entity_validation=True)
    
    print(f"Traitement en cours des {len(level0_rejected_data)} résumés rejetés Niveau 0...")
    
    start_time = time.time()
    
    # Traitement batch manuel
    results_rejected = []
    valid_rejected = []

    for i, summary_data in enumerate(level0_rejected_data):
        # Préparer les métadonnées
        metadata = {
            'id': summary_data['id'],
            'coherence_score': summary_data['coherence'],
            'factuality_score': summary_data['factuality'],
            'quality_grade': summary_data['quality_grade']
        }
        
        # Analyser le résumé
        result = detector_rejected.analyze_summary(summary_data['text'], metadata)
        results_rejected.append(result)
        
        # Ajouter aux valides si pas suspect
        if not result.is_suspect:
            valid_rejected.append(summary_data)
        
        # Progress indicator
        if (i + 1) % 50 == 0:
            print(f"   Traité {i + 1}/{len(level0_rejected_data)} résumés...")
    
    processing_time = time.time() - start_time
    
    print(f"Traitement terminé en {processing_time:.2f}s")
    print(f"\nRésultats sur résumés rejetés Niveau 0 (APRÈS CORRECTIONS):")
    print(f"   Total analysé: {len(level0_rejected_data)}")
    print(f"   Suspects détectés: {len(level0_rejected_data) - len(valid_rejected)}")
    print(f"   Taux de détection: {(len(level0_rejected_data) - len(valid_rejected))/len(level0_rejected_data)*100:.1f}%")
    print(f"   Temps moyen: {processing_time/len(level0_rejected_data)*1000:.1f}ms par résumé")
    
    # Distribution des grades pour les rejetés (NOUVELLES DONNÉES)
    print(f"\nDistribution par grade (résumés rejetés Niveau 0) - APRÈS CORRECTIONS:")
    rejected_grades = Counter([s['quality_grade'] for s in level0_rejected_data])
    for grade, count in sorted(rejected_grades.items()):
        suspect_count = sum(1 for i, s in enumerate(level0_rejected_data) 
                           if s['quality_grade'] == grade and results_rejected[i].is_suspect)
        avg_confidence = sum(r.confidence_score for i, r in enumerate(results_rejected) 
                           if level0_rejected_data[i]['quality_grade'] == grade) / max(1, count)
        print(f"   Grade {grade}: {suspect_count}/{count} suspects ({suspect_count/count*100:.1f}%) - Confiance: {avg_confidence:.3f}")
    
    # Sauvegarder pour usage ultérieur
    globals()['level0_rejected_results'] = results_rejected
    print(f"\nRésultats sauvegardés avec les nouvelles données")
    
else:
    print("Aucun résumé rejeté par Niveau 0 à analyser")
    results_rejected = []
    globals()['level0_rejected_results'] = []

Traitement en cours des 183 résumés rejetés Niveau 0...
   Traité 50/183 résumés...
   Traité 100/183 résumés...
   Traité 150/183 résumés...
Traitement terminé en 30.71s

Résultats sur résumés rejetés Niveau 0 (APRÈS CORRECTIONS):
   Total analysé: 183
   Suspects détectés: 181
   Taux de détection: 98.9%
   Temps moyen: 167.8ms par résumé

Distribution par grade (résumés rejetés Niveau 0) - APRÈS CORRECTIONS:
   Grade A+: 0/2 suspects (0.0%) - Confiance: 0.850
   Grade B: 4/4 suspects (100.0%) - Confiance: 0.000
   Grade B+: 119/119 suspects (100.0%) - Confiance: 0.003
   Grade C: 17/17 suspects (100.0%) - Confiance: 0.000
   Grade D: 41/41 suspects (100.0%) - Confiance: 0.002

Résultats sauvegardés avec les nouvelles données


In [17]:
# Corrélation avec grades de qualité
print(f"\nCorrélation avec grades de qualité:")
for grade in ['A+', 'A', 'B+', 'B', 'C', 'D']:
    grade_data = [s for s in level0_validated_data if s['quality_grade'] == grade]
    if grade_data:
        grade_results = [all_results_main[i] for i, s in enumerate(level0_validated_data) if s['quality_grade'] == grade]
        suspect_count = sum(1 for r in grade_results if r.is_suspect)  # CORRIGÉ: is_suspect
        total_count = len(grade_data)
        avg_confidence = sum(r.confidence_score for r in grade_results) / len(grade_results)
        print(f"   Grade {grade}: {suspect_count}/{total_count} suspects ({suspect_count/total_count*100:.1f}%) - Confiance moy: {avg_confidence:.3f}")

# Sauvegarder pour usage ultérieur
globals()['level0_validated_results'] = all_results_main


Corrélation avec grades de qualité:
   Grade A+: 1/58 suspects (1.7%) - Confiance moy: 0.873
   Grade A: 2/62 suspects (3.2%) - Confiance moy: 0.837
   Grade B+: 3/39 suspects (7.7%) - Confiance moy: 0.763
   Grade B: 3/7 suspects (42.9%) - Confiance moy: 0.521
   Grade D: 7/23 suspects (30.4%) - Confiance moy: 0.765


In [18]:
# ===== VUE D'ENSEMBLE AVEC NOUVELLES DONNÉES =====

# Combiner tous les nouveaux résultats
all_372_summaries = level0_validated_data + level0_rejected_data
all_372_results = level0_validated_results + level0_rejected_results

print(f"RÉSULTATS GLOBAUX (APRÈS CORRECTIONS):")
total_suspects_new = sum(1 for r in all_372_results if r.is_suspect)  # CORRIGÉ: is_suspect
detection_rate_new = total_suspects_new/len(all_372_results)*100

print(f"   Total analysé: {len(all_372_summaries)}")
print(f"   Suspects détectés: {total_suspects_new}")
print(f"   Taux de détection global: {detection_rate_new:.1f}%")

print(f"\nCOMPARAISON TAUX DE DÉTECTION:")
print(f"   AVANT corrections: 62.9%")
print(f"   APRÈS corrections: {detection_rate_new:.1f}%")
print(f"   Amélioration: {62.9 - detection_rate_new:+.1f} points")

print(f"\nRÉPARTITION PAR POPULATION (NOUVELLES DONNÉES):")
validated_suspects_new = len(level0_validated_data) - len(valid_summaries_main)
rejected_suspects_new = len(level0_rejected_data) - len(valid_rejected) if len(level0_rejected_data) > 0 else 0

print(f"   Population validée Niveau 0 ({len(level0_validated_data)} résumés):")
print(f"     Suspects détectés: {validated_suspects_new} ({validated_suspects_new/len(level0_validated_data)*100:.1f}%)")

if len(level0_rejected_data) > 0:
    print(f"   Population rejetée Niveau 0 ({len(level0_rejected_data)} résumés):")
    print(f"     Suspects détectés: {rejected_suspects_new} ({rejected_suspects_new/len(level0_rejected_data)*100:.1f}%)")

print(f"\nDISTRIBUTION GLOBALE PAR GRADE (NOUVELLES DONNÉES):")
print(f"{'Grade':<8} {'Total':<8} {'Suspects':<12} {'%':<8} {'Confiance':<12}")
print("-" * 55)

for grade in ['A+', 'A', 'B+', 'B', 'C', 'D']:
    grade_summaries = [s for s in all_372_summaries if s['quality_grade'] == grade]
    if grade_summaries:
        grade_results = [all_372_results[i] for i, s in enumerate(all_372_summaries) if s['quality_grade'] == grade]
        suspect_count = sum(1 for r in grade_results if r.is_suspect)  # CORRIGÉ: is_suspect
        total_count = len(grade_summaries)
        avg_confidence = sum(r.confidence_score for r in grade_results) / len(grade_results)
        print(f"{grade:<8} {total_count:<8} {suspect_count:<12} {suspect_count/total_count*100:<8.1f} {avg_confidence:.3f}")

print(f"\nPERFORMANCE GLOBALE (NOUVELLES DONNÉES):")
all_processing_times_new = [r.processing_time_ms for r in all_372_results]
avg_time_new = sum(all_processing_times_new) / len(all_processing_times_new)
max_time_new = max(all_processing_times_new)
target_100ms_global_new = sum(1 for t in all_processing_times_new if t < 100) / len(all_processing_times_new) * 100

print(f"   Temps moyen: {avg_time_new:.1f}ms (vs 116.3ms avant)")
print(f"   Temps maximum: {max_time_new:.1f}ms (vs 388.6ms avant)")
print(f"   Objectif <100ms: {target_100ms_global_new:.1f}% (vs 49.5% avant)")

# Enrichissement global
total_fact_check_global = sum(len(r.fact_check_candidates) for r in all_372_results)

# Wikidata queries - vérification sécurisée
total_wikidata_global = 0
for r in all_372_results:
    if 'validation_hints' in r.enrichment_metadata:
        hints = r.enrichment_metadata['validation_hints']
        if isinstance(hints, dict) and 'wikidata_queries' in hints:
            total_wikidata_global += len(hints['wikidata_queries'])

print(f"   Candidats fact-check: {total_fact_check_global} (vs 5 avant)")
print(f"   Requêtes Wikidata: {total_wikidata_global} (vs 5 avant)")

print(f"\n{'='*60}")
print(f"SUCCÈS DES CORRECTIONS:")
improvements = []
if detection_rate_new < 50: improvements.append("✓ Taux détection réduit")
if target_100ms_global_new > 60: improvements.append("✓ Performance améliorée") 
if total_fact_check_global > 50: improvements.append("✓ Enrichissement augmenté")
print(f"   {len(improvements)}/3 objectifs atteints")
print(f"   {' | '.join(improvements) if improvements else 'Ajustements nécessaires'}")
print(f"{'='*60}")

RÉSULTATS GLOBAUX (APRÈS CORRECTIONS):
   Total analysé: 372
   Suspects détectés: 197
   Taux de détection global: 53.0%

COMPARAISON TAUX DE DÉTECTION:
   AVANT corrections: 62.9%
   APRÈS corrections: 53.0%
   Amélioration: +9.9 points

RÉPARTITION PAR POPULATION (NOUVELLES DONNÉES):
   Population validée Niveau 0 (189 résumés):
     Suspects détectés: 16 (8.5%)
   Population rejetée Niveau 0 (183 résumés):
     Suspects détectés: 181 (98.9%)

DISTRIBUTION GLOBALE PAR GRADE (NOUVELLES DONNÉES):
Grade    Total    Suspects     %        Confiance   
-------------------------------------------------------
A+       60       1            1.7      0.873
A        62       2            3.2      0.837
B+       158      122          77.2     0.191
B        11       7            63.6     0.332
C        17       17           100.0    0.000
D        64       48           75.0     0.277

PERFORMANCE GLOBALE (NOUVELLES DONNÉES):
   Temps moyen: 103.1ms (vs 116.3ms avant)
   Temps maximum: 553.4ms (

In [19]:
if len(level0_rejected_data) > 0:
    detector_rejected = HeuristicAnalyzer(enable_wikidata=False, enable_entity_validation=True)
    
    print(f"Traitement en cours des {len(level0_rejected_data)} résumés rejetés Niveau 0...")
    
    start_time = time.time()
    
    # Traitement batch manuel
    results_rejected = []
    valid_rejected = []

    for i, summary_data in enumerate(level0_rejected_data):
        # Préparer les métadonnées
        metadata = {
            'id': summary_data['id'],
            'coherence_score': summary_data['coherence'],
            'factuality_score': summary_data['factuality'],
            'quality_grade': summary_data['quality_grade']
        }
        
        # Analyser le résumé
        result = detector_rejected.analyze_summary(summary_data['text'], metadata)
        results_rejected.append(result)
        
        # Ajouter aux valides si pas suspect
        if not result.is_suspect:
            valid_rejected.append(summary_data)
    
    processing_time = time.time() - start_time
    
    print(f"Traitement terminé en {processing_time:.2f}s")
    print(f"\nRésultats sur résumés rejetés Niveau 0:")
    print(f"   Total analysé: {len(level0_rejected_data)}")
    print(f"   Suspects détectés: {len(level0_rejected_data) - len(valid_rejected)}")
    print(f"   Taux de détection: {(len(level0_rejected_data) - len(valid_rejected))/len(level0_rejected_data)*100:.1f}%")
    print(f"   Temps moyen: {processing_time/len(level0_rejected_data)*1000:.1f}ms par résumé")
    
    # Distribution des grades pour les rejetés
    print(f"\nDistribution par grade (résumés rejetés Niveau 0):")
    rejected_grades = Counter([s['quality_grade'] for s in level0_rejected_data])
    for grade, count in sorted(rejected_grades.items()):
        suspect_count = sum(1 for i, s in enumerate(level0_rejected_data) 
                           if s['quality_grade'] == grade and results_rejected[i].is_suspect)  # CORRIGÉ: is_suspect
        print(f"   Grade {grade}: {suspect_count}/{count} suspects ({suspect_count/count*100:.1f}%)")
    
    # Comparaison des raisons de détection
    print(f"\nAnalyse des patterns de détection:")
    rejected_issues = []
    for result in results_rejected:
        # Utiliser 'issues' au lieu de 'detected_issues'
        rejected_issues.extend([str(issue) for issue in result.issues])
    
    if rejected_issues:
        issue_types = Counter([issue.split(':')[0] if ':' in str(issue) else str(issue)[:20] for issue in rejected_issues])
        print(f"   Types d'issues les plus fréquents (résumés rejetés Niveau 0):")
        for issue_type, count in issue_types.most_common(5):
            print(f"     {issue_type}: {count} cas")
    else:
        print(f"   Aucune issue spécifique détectée dans les résultats")
    
    # Sauvegarder pour usage ultérieur
    globals()['level0_rejected_results'] = results_rejected
    print(f"\nRésultats sauvegardés pour analyse ultérieure")
    
else:
    print("Aucun résumé rejeté par Niveau 0 à analyser")
    results_rejected = []
    globals()['level0_rejected_results'] = []

Traitement en cours des 183 résumés rejetés Niveau 0...
Traitement terminé en 21.81s

Résultats sur résumés rejetés Niveau 0:
   Total analysé: 183
   Suspects détectés: 181
   Taux de détection: 98.9%
   Temps moyen: 119.2ms par résumé

Distribution par grade (résumés rejetés Niveau 0):
   Grade A+: 0/2 suspects (0.0%)
   Grade B: 4/4 suspects (100.0%)
   Grade B+: 119/119 suspects (100.0%)
   Grade C: 17/17 suspects (100.0%)
   Grade D: 41/41 suspects (100.0%)

Analyse des patterns de détection:
   Types d'issues les plus fréquents (résumés rejetés Niveau 0):
     {'type': 899 cas

Résultats sauvegardés pour analyse ultérieure


## 7. Vue d'Ensemble - Cartographie Complète des 372 Résumés

In [20]:
# Combiner tous les résultats
all_372_summaries = level0_validated_data + level0_rejected_data
all_372_results = level0_validated_results + level0_rejected_results

print(f"RÉSULTATS GLOBAUX:")
print(f"   Total analysé: {len(all_372_summaries)}")
print(f"   Suspects détectés: {sum(1 for r in all_372_results if r.is_suspect)}")  # CORRIGÉ: is_suspect
print(f"   Taux de détection global: {sum(1 for r in all_372_results if r.is_suspect)/len(all_372_results)*100:.1f}%")  # CORRIGÉ: is_suspect

print(f"\nRÉPARTITION PAR POPULATION:")
validated_suspects = len(level0_validated_data) - len(valid_summaries_main)
rejected_suspects = len(level0_rejected_data) - len(valid_rejected) if len(level0_rejected_data) > 0 else 0

print(f"   Population validée Niveau 0 ({len(level0_validated_data)} résumés):")
print(f"     Suspects détectés: {validated_suspects} ({validated_suspects/len(level0_validated_data)*100:.1f}%)")

if len(level0_rejected_data) > 0:
    print(f"   Population rejetée Niveau 0 ({len(level0_rejected_data)} résumés):")
    print(f"     Suspects détectés: {rejected_suspects} ({rejected_suspects/len(level0_rejected_data)*100:.1f}%)")

print(f"\nDISTRIBUTION GLOBALE PAR GRADE:")
for grade in ['A+', 'A', 'B+', 'B', 'C', 'D']:
    grade_summaries = [s for s in all_372_summaries if s['quality_grade'] == grade]
    if grade_summaries:
        grade_results = [all_372_results[i] for i, s in enumerate(all_372_summaries) if s['quality_grade'] == grade]
        suspect_count = sum(1 for r in grade_results if r.is_suspect)  # CORRIGÉ: is_suspect
        total_count = len(grade_summaries)
        avg_confidence = sum(r.confidence_score for r in grade_results) / len(grade_results)
        print(f"   Grade {grade}: {suspect_count}/{total_count} suspects ({suspect_count/total_count*100:.1f}%) - Confiance: {avg_confidence:.3f}")

print(f"\nPERFORMANCE GLOBALE:")
all_processing_times = [r.processing_time_ms for r in all_372_results]
avg_time = sum(all_processing_times) / len(all_processing_times)
max_time = max(all_processing_times)
target_100ms_global = sum(1 for t in all_processing_times if t < 100) / len(all_processing_times) * 100

print(f"   Temps moyen: {avg_time:.1f}ms")
print(f"   Temps maximum: {max_time:.1f}ms")
print(f"   Objectif <100ms: {target_100ms_global:.1f}% des résumés")

RÉSULTATS GLOBAUX:
   Total analysé: 372
   Suspects détectés: 197
   Taux de détection global: 53.0%

RÉPARTITION PAR POPULATION:
   Population validée Niveau 0 (189 résumés):
     Suspects détectés: 16 (8.5%)
   Population rejetée Niveau 0 (183 résumés):
     Suspects détectés: 181 (98.9%)

DISTRIBUTION GLOBALE PAR GRADE:
   Grade A+: 1/60 suspects (1.7%) - Confiance: 0.873
   Grade A: 2/62 suspects (3.2%) - Confiance: 0.837
   Grade B+: 122/158 suspects (77.2%) - Confiance: 0.191
   Grade B: 7/11 suspects (63.6%) - Confiance: 0.332
   Grade C: 17/17 suspects (100.0%) - Confiance: 0.000
   Grade D: 48/64 suspects (75.0%) - Confiance: 0.277

PERFORMANCE GLOBALE:
   Temps moyen: 79.5ms
   Temps maximum: 308.7ms
   Objectif <100ms: 71.8% des résumés


In [21]:
print(f"   Enrichissement de {len(all_372_summaries)} résumés avec 6 champs utiles")
print(f"   Performance <100ms respectée pour {target_100ms_global:.1f}% des cas")

# Sauvegarder les variables globales pour utilisation ultérieure
globals()['all_372_summaries'] = all_372_summaries
globals()['all_372_results'] = all_372_results

   Enrichissement de 372 résumés avec 6 champs utiles
   Performance <100ms respectée pour 71.8% des cas


## 8. Analyse des Patterns d'Enrichissement

In [22]:
# Analyse des nouvelles informations enrichies
print("ANALYSE DES PATTERNS D'ENRICHISSEMENT\n")

# Échantillon représentatif pour analyse détaillée
sample_size = min(20, len(all_372_results))
sample_indices = np.linspace(0, len(all_372_results)-1, sample_size, dtype=int)

print(f"Analyse sur échantillon de {sample_size} résumés:")

# Analyse des métriques disponibles
word_counts = []
confidence_scores = []
entity_counts = []
priority_scores = []

for i in sample_indices:
    result = all_372_results[i]
    summary = all_372_summaries[i]
    
    # Collecter les métriques réellement disponibles
    word_counts.append(result.word_count)  # Propriété directe
    confidence_scores.append(result.confidence_score)  # Propriété directe
    entity_counts.append(result.entities_detected)  # Propriété directe
    priority_scores.append(result.priority_score)  # Propriété directe
    
    # Afficher des détails pour les premiers résultats
    if i < 3:
        print(f"\nÉchantillon {i+1}:")
        print(f"   ID: {summary['id']} (Grade: {summary['quality_grade']})")
        print(f"   Mots: {result.word_count}")
        print(f"   Confiance: {result.confidence_score:.3f}")
        print(f"   Entités: {result.entities_detected}")
        print(f"   Score priorité: {result.priority_score:.3f}")
        print(f"   Statut: {'SUSPECT' if result.is_suspect else 'VALIDE'}")
        print(f"   Temps: {result.processing_time_ms:.1f}ms")

ANALYSE DES PATTERNS D'ENRICHISSEMENT

Analyse sur échantillon de 20 résumés:

Échantillon 1:
   ID: 0_adaptive (Grade: A)
   Mots: 85
   Confiance: 0.750
   Entités: 2
   Score priorité: 0.000
   Statut: VALIDE
   Temps: 89.4ms


In [23]:
print(f"\nMétriques d'enrichissement:")
print(f"   Longueur moyenne: {np.mean(word_counts):.1f} mots (min: {np.min(word_counts)}, max: {np.max(word_counts)})")
print(f"   Score de confiance moyen: {np.mean(confidence_scores):.3f}")  # CORRIGÉ: confidence_scores
print(f"   Entités détectées en moyenne: {np.mean(entity_counts):.1f}")  # CORRIGÉ: entity_counts
print(f"   Score de priorité moyen: {np.mean(priority_scores):.3f}")


Métriques d'enrichissement:
   Longueur moyenne: 229.8 mots (min: 21, max: 706)
   Score de confiance moyen: 0.423
   Entités détectées en moyenne: 15.5
   Score de priorité moyen: 0.520


In [24]:
# Analyse des candidats fact-check
all_fact_check_candidates = []
for result in all_372_results:
    all_fact_check_candidates.extend(result.fact_check_candidates)

if all_fact_check_candidates:
    candidate_types = Counter([c.get('type', 'unknown') if isinstance(c, dict) else str(c) for c in all_fact_check_candidates])
    print(f"\nTypes de candidats fact-check identifiés:")
    for ctype, count in candidate_types.most_common(5):
        print(f"   {ctype}: {count} candidats ({count/len(all_fact_check_candidates)*100:.1f}%)")
else:
    print(f"\nAucun candidat fact-check trouvé dans les résultats")


Types de candidats fact-check identifiés:
   entity_verification: 356 candidats (86.6%)
   factual_claim: 55 candidats (13.4%)


In [25]:
# ===== SAUVEGARDE RÉSULTATS SIMPLIFIÉE =====

# Créer le DataFrame avec les données réellement disponibles
simple_results_df = pd.DataFrame([
    {
        'id': all_372_summaries[i]['id'],
        'level0_status': 'validated' if i < len(level0_validated_data) else 'rejected',
        'original_grade': all_372_summaries[i]['quality_grade'],
        'coherence': all_372_summaries[i]['coherence'],
        'factuality': all_372_summaries[i]['factuality'],
        'is_suspect': all_372_results[i].is_suspect,  # CORRIGÉ: is_suspect
        'confidence_score': all_372_results[i].confidence_score,
        'risk_level': all_372_results[i].risk_level,
        'processing_time_ms': all_372_results[i].processing_time_ms,
        'word_count': all_372_results[i].word_count,  # Propriété directe
        'entities_detected': all_372_results[i].entities_detected,  # Propriété directe
        'suspicious_entities': all_372_results[i].suspicious_entities,  # Propriété directe
        'fact_check_candidates_count': len(all_372_results[i].fact_check_candidates),
        'priority_score': all_372_results[i].priority_score,  # Propriété directe
        'num_issues': len(all_372_results[i].issues)  # Propriété correcte
    }
    for i in range(len(all_372_summaries))
])

# Créer le répertoire de sortie
output_path = os.path.join(project_root, 'data', 'detection')
os.makedirs(output_path, exist_ok=True)

# Sauvegarder les résultats simplifiés
results_file = os.path.join(output_path, 'level1_heuristic_results.csv')
simple_results_df.to_csv(results_file, index=False)

# Statistiques simplifiées
simple_stats = {
    'total_analyzed': len(all_372_summaries),
    'level0_validated': len(level0_validated_data),
    'level0_rejected': len(level0_rejected_data),
    'total_suspects': sum(1 for r in all_372_results if r.is_suspect),
    'detection_rate_percent': sum(1 for r in all_372_results if r.is_suspect)/len(all_372_results)*100,
    'avg_time_ms': sum(r.processing_time_ms for r in all_372_results) / len(all_372_results),
    'max_time_ms': max(r.processing_time_ms for r in all_372_results),
    'total_fact_check_candidates': sum(len(r.fact_check_candidates) for r in all_372_results)
}

stats_file = os.path.join(output_path, 'level1_heuristic_stats.json')
with open(stats_file, 'w', encoding='utf-8') as f:
    json.dump(simple_stats, f, indent=2, ensure_ascii=False)

print(f"=== FICHIERS SAUVEGARDÉS (VERSION SIMPLIFIÉE) ===")
print(f"Résultats: {results_file}")
print(f"Statistiques: {stats_file}")
print(f"{len(simple_results_df)} résumés avec {len(simple_results_df.columns)} colonnes")
print(f"Taux de détection: {simple_stats['detection_rate_percent']:.1f}%")
print(f"Performance moyenne: {simple_stats['avg_time_ms']:.1f}ms")
print(f"Candidats fact-check: {simple_stats['total_fact_check_candidates']}")

=== FICHIERS SAUVEGARDÉS (VERSION SIMPLIFIÉE) ===
Résultats: c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\data\detection\level1_heuristic_results.csv
Statistiques: c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\data\detection\level1_heuristic_stats.json
372 résumés avec 15 colonnes
Taux de détection: 53.0%
Performance moyenne: 79.5ms
Candidats fact-check: 411


In [26]:
# CELLULE DÉSACTIVÉE - Nécessite des propriétés non disponibles
# (get_priority_score, detected_issues)

print("Cellule désactivée car elle utilise des propriétés inexistantes :")
print("- result.get_priority_score() n'existe pas")
print("- result.detected_issues n'existe pas")
print("\nPour voir les candidats prioritaires, utilisez :")
print("- result.priority_score (propriété directe)")  
print("- result.issues (propriété correcte)")
print("- Tri manuel basé sur confidence_score et risk_level")

# Alternative basique fonctionnelle :
print(f"\nTOP 5 résumés par confiance faible (plus suspects) :")
confidence_ranking = [(i, r.confidence_score, r.risk_level) for i, r in enumerate(all_372_results)]
confidence_ranking.sort(key=lambda x: x[1])  # Plus faible confiance = plus suspect

for rank, (idx, conf, risk) in enumerate(confidence_ranking[:5], 1):
    summary = all_372_summaries[idx]
    result = all_372_results[idx]
    print(f"   {rank}. {summary['id']} (Grade: {summary['quality_grade']})")
    print(f"      Confiance: {conf:.3f} | Risque: {risk} | Issues: {len(result.issues)}")

Cellule désactivée car elle utilise des propriétés inexistantes :
- result.get_priority_score() n'existe pas
- result.detected_issues n'existe pas

Pour voir les candidats prioritaires, utilisez :
- result.priority_score (propriété directe)
- result.issues (propriété correcte)
- Tri manuel basé sur confidence_score et risk_level

TOP 5 résumés par confiance faible (plus suspects) :
   1. 6_confidence_weighted (Grade: B+)
      Confiance: 0.000 | Risque: critical | Issues: 5
   2. 66_confidence_weighted (Grade: B)
      Confiance: 0.000 | Risque: critical | Issues: 6
   3. 112_confidence_weighted (Grade: D)
      Confiance: 0.000 | Risque: critical | Issues: 4
   4. 161_confidence_weighted (Grade: B)
      Confiance: 0.000 | Risque: critical | Issues: 6
   5. 168_confidence_weighted (Grade: B+)
      Confiance: 0.000 | Risque: critical | Issues: 5


## 9. Sauvegarde des Résultats Enrichis

In [29]:
# Affichage des fichiers sauvegardés (référence aux variables de la cellule 31)
print(f"Fichiers sauvegardés:")
print(f"   Résultats: {results_file}")
print(f"   Statistiques: {stats_file}")

print(f"   {len(simple_results_df)} résumés avec {len(simple_results_df.columns)} colonnes de données")
print(f"   Performance moyenne: {simple_stats['avg_time_ms']:.1f}ms")
print(f"   Taux de détection global: {simple_stats['detection_rate_percent']:.1f}%")
print(f"   Candidats fact-check: {simple_stats['total_fact_check_candidates']}")


Fichiers sauvegardés:
   Résultats: c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\data\detection\level1_heuristic_results.csv
   Statistiques: c:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\data\detection\level1_heuristic_stats.json
   372 résumés avec 15 colonnes de données
   Performance moyenne: 79.5ms
   Taux de détection global: 53.0%
   Candidats fact-check: 411
